In [1]:
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, mean_absolute_error

import tensorflow.keras.backend as K

from tensorflow.keras import metrics
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
#sepearte features
features = df.drop(columns = "output", axis = 1)

#seperate output
labels = df['output'].values

#scale the data

#make the scaler object
scaler = MinMaxScaler()

#scale the data
scaler.fit(features)

#then reset the features as the scaled features
features = scaler.transform(features)

No clue what this is, is that it gets all of the output numbers which are going to be 0,1

In [4]:
y_class = LabelEncoder().fit_transform(labels)
y_class

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

then we get the unique ones which will give us to the total output categorical amount in this case it is binary

In [5]:
n_class = len(np.unique(y_class))
n_class

2

In [6]:
#this will give us the number of features
n_features = features.shape[1]
n_features

13

In [7]:
#now split the data test size = 0.33
X_train, X_test, y_train, y_test, y_train_class, y_test_class = train_test_split(features, labels, y_class, test_size=0.33, random_state=1)

In [8]:
#making a function but not sure what it is
def soft_acc(y_true, y_pred):
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

In [9]:
#this looks like it is making the model
visible = Input(shape = (n_features))
print(visible)

KerasTensor(type_spec=TensorSpec(shape=(None, 13), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [10]:
#then start with the layers

#we have to feed visible 
hidden1 = Dense(32, activation = "relu", kernel_initializer = 'he_normal')(visible)

#then we have to feed in hidden1
hidden2 = Dense(16, activation = "relu", kernel_initializer = "he_normal")(hidden1)

#regression
out_reg = Dense(1, activation = "linear")(hidden2)

#classification output
out_clas = Dense(n_class, activation = "softmax")(hidden2)

#define model
model = Model(inputs = visible, outputs = [out_reg, out_clas])

model.compile(loss = ['mse', 'sparse_categorical_crossentropy'], metrics = ['soft_acc', metrics.SparseCategoricalAccuracy()], optimizer = 'Adam')

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           448         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 16)           528         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1)            17          dense_1[0][0]                    
______________________________________________________________________________________________

In [12]:
history = model.fit(X_train, [y_train, y_train_class], epochs = 20, batch_size = 32, verbose = 0, validation_data = (X_test, [y_test, y_test_class]))

ValueError: in user code:

    C:\Users\Diego\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\Diego\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Diego\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Diego\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Diego\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Diego